In [37]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

In [38]:
#read the model
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1, 4), stride=(1, 2))
        self.bn1 = nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4))
        self.dropout1 = nn.Dropout(p=0.25)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 2), stride=(1, 2))
        self.bn2 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4))
        self.dropout2 = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(48576, 128)
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 6)


    def forward(self, x):
        x = torch.unsqueeze(x, 1)
        #print('x:', x.shape)
        x = self.conv1(x)
        #print('conv1:', x.shape)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        #print('conv2:', x.shape)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        #print('flatten:', x.shape)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

model = EEGNet()

In [43]:
import numpy as np

def sliding_window(signal, window_size, step_size):
    n_channels, n_samples = signal.shape
    n_windows = int((n_samples - window_size) / step_size) + 1
    windows = np.zeros((n_channels, window_size, n_windows))
    for i in range(n_windows):
        windows[:, :,i ] = signal.iloc[:, i*step_size:i*step_size+window_size]
    return windows

signal = pd.read_csv("C:\\Users\\a1882\Desktop\EEG\eegdata\\raw\lefthand_zyy_05_epocflex_2023.03.22t16.50.54+08.00.md.bp.csv", header=None)
signal = pd.DataFrame(signal)
# 定义滑窗大小和滑动步长
window_size = 3000
step_size = 100

# 对信号进行滑窗处理
windows = sliding_window(signal, window_size, step_size)

# 输出滑窗后的信号形状
print("滑窗后信号形状：", windows.shape)
#print(windows)

滑窗后信号形状： (33, 3000, 126)


In [44]:
import onnx
import onnxruntime
save_path = './cnn_3000_30e_26.pt'

# onnx_file_name = "./cnn_3000_100e_100.onnx"
torch.load(save_path, map_location=torch.device('cpu'))

OrderedDict([('conv1.weight',
              tensor([[[[-0.3445,  0.3977, -0.2108, -0.3395]]],
              
              
                      [[[ 0.0366,  0.3111, -0.0930, -0.2412]]],
              
              
                      [[[-0.4966, -0.4928,  0.2660, -0.0154]]],
              
              
                      [[[-0.4080,  0.4452, -0.2056, -0.0517]]],
              
              
                      [[[-0.3918, -0.3286, -0.3431, -0.3232]]],
              
              
                      [[[ 0.0481,  0.0398,  0.3754,  0.2612]]],
              
              
                      [[[-0.3075, -0.3402,  0.4785,  0.1673]]],
              
              
                      [[[ 0.3987,  0.2521,  0.4194, -0.1689]]],
              
              
                      [[[-0.2991, -0.3978, -0.2616, -0.3489]]],
              
              
                      [[[-0.4447,  0.2200, -0.4190,  0.0629]]],
              
              
                      [[[ 0.32

In [45]:
import onnx
# 我们可以使用异常处理的方法进行检验
try:
    # 当我们的模型不可用时，将会报出异常
    onnx.checker.check_model(onnx_file_name)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s"%e)
else:
    # 模型可用时，将不会报出异常，并会输出“The model is valid!”
    print("The model is valid!")

The model is valid!


In [46]:
model = EEGNet()
model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
model.eval()
for i in range(windows.shape[2]):
    signal = pd.DataFrame(windows[:,:,i])
    input = torch.from_numpy(signal.values)
    input = input.unsqueeze(0)
    output = model(input.float())

    # # print(output)
    # probabilities = torch.nn.functional.softmax(output, dim=1)
    # print(probabilities)

    _, pred = torch.max(output, dim=1)  # 找到预测分数最大的类别，得到预测类别
    label_map = {0: 'lefthand', 1:'read' ,  2:'rest', 3: 'walkbase', 4: 'walkl' ,5: 'walkfocus'}
    print(label_map[pred.item()])

lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
rest
lefthand
lefthand
lefthand
rest
lefthand
lefthand
lefthand
rest
rest
lefthand
lefthand
rest
rest
lefthand
lefthand
rest
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
rest
rest
lefthand
lefthand
lefthand
rest
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefth

In [12]:
    #print(output)
    # probabilities = torch.nn.functional.softmax(output, dim=1)
    # print(probabilities)

In [49]:
model = EEGNet()
model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
model.eval()
probabilities = []
for i in range(windows.shape[2]):
    signal = pd.DataFrame(windows[:,:,i])
    input = torch.from_numpy(signal.values)
    input = input.unsqueeze(0)
    output = model(input.float())

    # # print(output)
    #probabilitie = torch.nn.functional.softmax(output, dim=1)
    # print(probabilities)
    #probabilities.append(probabilitie)
    probabilities.append(output)
    if len(probabilities) == 5:
        average_predict = sum(probabilities)/5
        _, pred = torch.max(average_predict, dim=1)  # 找到预测分数最大的类别，得到预测类别
        label_map = {0: 'lefthand', 1:'read' ,  2:'rest', 3: 'walkbase', 4: 'walkl' ,5: 'walkfocus'}
        print(label_map[pred.item()])
        probabilities = []
    #


lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
lefthand
